In [ ]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload

# import own functions
from utils import find_folders

### Add Directories/ Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
##LOAD NEW FILES
subID = 'Sub028'

fft110_fname = subID + '_FFT_110.npy'
fft125_fname = subID + '_FFT_125.npy'
fft145_fname = subID + '_FFT_145.npy'

fnames_path = os.path.join(project_path,
    'data',
    'switching_freqs',
    subID)

fft110 = np.load(os.path.join(
    fnames_path,
    fft110_fname
))

fft125 = np.load(os.path.join(
    fnames_path,
    fft125_fname
))

fft145 = np.load(os.path.join(
    fnames_path,
    fft145_fname
))

In [ ]:
### PLOT SPECTROGRAMS
%matplotlib qt

fig = plt.figure(figsize = (9,4))
ax = [plt.subplot(1,3,i+1) for i in range(3)]

for a in ax[1:3]:
    a.set_yticklabels([])
for a in ax:  a.set_xlabel('Time [sec]')
#plt.subplots_adjust(wspace = 1, hspace = 0)

tp1 = 1
tp2 = 620
tp3 = 146

ax[0].pcolormesh(fft110[1,:,tp1:tp1+15], cmap = 'viridis', vmin = 0, vmax = 0.2)
ax[0].set_ylabel('Frequency [Hz]')
ax[0].set_title('Stim 110Hz')
ax[1].pcolormesh(fft125[1,:,tp2:tp2+15], cmap = 'viridis', vmin = 0, vmax = 0.2)
ax[1].set_title('Stim 125Hz')
ax[2].pcolormesh(fft145[1,:,tp3:tp3+15], cmap = 'viridis', vmin = 0, vmax = 0.2)
ax[2].set_title('Stim 145Hz')


for a in ax:
    a.set_ylim(40,100)

In [ ]:
plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig'
),
dpi = 400)

plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig.pdf'
))

In [ ]:
### PLOT POWER SPECTRA
tp1 = 16
tp2 = 948
tp3 = 262
%matplotlib qt 
fig, axes = plt.subplots(1,1, figsize = (4,4))

side = 1

ps_110 = np.mean(fft110[side,:,tp1:tp1+15],1)
ps_125 = np.mean(fft125[side,:,tp2:tp2+15],1)
ps_145 = np.mean(fft145[side,:,tp3:tp3+15],1)


plt.plot(np.arange(1,127),ps_110,label = 'Stim 110Hz', lw = 2, color = 'turquoise')
plt.plot(np.arange(1,127),ps_125,label = 'Stim 125Hz', lw = 2, color = 'midnightblue')
plt.plot(np.arange(1,127),ps_145,label = 'Stim 145Hz', lw = 2, color = 'deeppink')
    
plt.xlim(50,80)
plt.ylim(0,0.10)
plt.xlabel('Frequency [Hz]')
#plt.ylabel('LFP Power')
    
plt.legend()
plt.show()

In [ ]:
plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig_PS'
),
dpi = 200)

plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig_PS.pdf'
))